In [2]:
import os
import glob
from datetime import datetime
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values

In [6]:
def get_most_recent_folder():
    parent_dir = './data/'
    folders = glob.glob(parent_dir + '*')
    folders.sort(key=os.path.getmtime, reverse=True)
    recent_folder = folders[0]
    return recent_folder

### Update `instructor` table

In [18]:
def csv_to_postgres_instructor():
    folder_path = get_most_recent_folder()
    df = pd.read_csv(f"{folder_path}/instructor.csv")
    df = df.replace({np.NaN: None})

    params = {
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': 54322
    }

    conn = psycopg2.connect(**params)
    cur = conn.cursor()

    query_template = sql.SQL("""
        INSERT INTO public.instructor (id, name, first_name, last_name, email)
        VALUES %s
        ON CONFLICT (id) DO UPDATE SET
            name = excluded.name,
            first_name = excluded.first_name,
            last_name = excluded.last_name,
            email = excluded.email
    """)

    records = df.to_records(index=False).tolist()
    execute_values(cur, query_template, records)
    conn.commit()
    cur.close()
    conn.close()

### Update `course` table

In [4]:
def csv_to_postgres_course():
    """
    take a csv file and insert into postgres
    or update if the `name` already exists
    """
    folder_path = get_most_recent_folder()
    df = pd.read_csv(f"{folder_path}/course.csv")
    df = df.replace({np.NaN: None})

    params = {
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': 54322
    }

    conn = psycopg2.connect(**params)
    cur = conn.cursor()

    query_template = sql.SQL("""
        INSERT INTO public.course (course_id, year, name, subject, title, description, units_min, units_max, repeatable, grading_basis, final_exam, academic_group, academic_org, way_a_ii, way_aqr, way_ce, way_edp, way_er, way_fr, way_sma, way_si)
        VALUES %s
        ON CONFLICT (name) DO UPDATE SET
            course_id = excluded.course_id,
            year = excluded.year,
            subject = excluded.subject,
            title = excluded.title,
            description = excluded.description,
            units_min = excluded.units_min,
            units_max = excluded.units_max,
            repeatable = excluded.repeatable,
            grading_basis = excluded.grading_basis,
            final_exam = excluded.final_exam,
            academic_group = excluded.academic_group,
            academic_org = excluded.academic_org,
            way_a_ii = excluded.way_a_ii,
            way_aqr = excluded.way_aqr,
            way_ce = excluded.way_ce,
            way_edp = excluded.way_edp,
            way_er = excluded.way_er,
            way_fr = excluded.way_fr,
            way_sma = excluded.way_sma,
            way_si = excluded.way_si
    """)

    records = df.to_records(index=False).tolist()
    execute_values(cur, query_template, records)
    conn.commit()
    cur.close()
    conn.close()

In [5]:
csv_to_postgres_course()

In [ ]:
# Untested new potential version to handle inserts, updates and then deletes
def csv_to_postgres_course_w_temp():
    folder_path = get_most_recent_folder()
    df = pd.read_csv(f"{folder_path}/course.csv")
    df = df.replace({np.NaN: None})

    params = {
        'dbname': 'postgres',
        'user': 'postgres',
        'password': 'postgres',
        'host': 'localhost',
        'port': 54322
    }

    conn = psycopg2.connect(**params)
    cur = conn.cursor()

    # create temporary table
    cur.execute("""
        CREATE TEMPORARY TABLE temp_course AS 
        SELECT * FROM course WITH NO DATA;
    """)

    # populate temporary table with CSV data
    execute_values(cur, sql.SQL("""
        INSERT INTO temp_course (course_id, year, name, subject, title, description, units_min, units_max, repeatable, grading_basis, final_exam, academic_group, academic_org, way_a_ii, way_aqr, way_ce, way_edp, way_er, way_fr, way_sma, way_si)
        VALUES %s
    """), df.to_records(index=False).tolist())
    
    # perform update/insert operations
    cur.execute("""
        INSERT INTO course
        SELECT * FROM temp_course
        ON CONFLICT (name) DO UPDATE SET
            course_id = excluded.course_id,
            year = excluded.year,
            subject = excluded.subject,
            title = excluded.title,
            description = excluded.description,
            units_min = excluded.units_min,
            units_max = excluded.units_max,
            repeatable = excluded.repeatable,
            grading_basis = excluded.grading_basis,
            final_exam = excluded.final_exam,
            academic_group = excluded.academic_group,
            academic_org = excluded.academic_org,
            way_a_ii = excluded.way_a_ii,
            way_aqr = excluded.way_aqr,
            way_ce = excluded.way_ce,
            way_edp = excluded.way_edp,
            way_er = excluded.way_er,
            way_fr = excluded.way_fr,
            way_sma = excluded.way_sma,
            way_si = excluded.way_si
    """)

    # delete rows in main table that do not exist in CSV
    cur.execute("""
        DELETE FROM course
        WHERE name NOT IN (SELECT name FROM temp_course)
    """)

    conn.commit()
    cur.close()
    conn.close()
